In [4]:
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import joblib

In [2]:
# Read data
# X: features, y: labels
features = pd.read_csv("../JeuDeDonnees/alt_acsincome_ca_features_85(1).csv")
labels = pd.read_csv("../JeuDeDonnees/alt_acsincome_ca_labels_85.csv")  

In [ ]:
# Print content
print("DATABASE 1")
print(features.head())
print(features.columns)

# Delete missing values - NULL, If 'MAR' or 'COW' contain NULL values 
features['AGEP'].hist(bins=30)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Nb de personnes')
plt.show()
# Limpiar los datos eliminando filas con valores nulos en 'MAR' y 'COW'
features_df = features.dropna(subset=['MAR', 'COW'])

# Create relation between 'MAR' and 'COW' 
features_relation = features_df.groupby(['MAR', 'COW']).size().reset_index(name='QUANTITÉ')
print(features_relation)

# Create Graphic
plt.figure(figsize=(10, 6))
sns.barplot(data=features_relation, x='MAR', y='QUANTITÉ', hue='COW', palette='viridis')
plt.title('Relation entre MAR (État Civil) et COW (Occupation)')
plt.xlabel('MAR (État Civil)')
plt.ylabel('QUANTITÉ PERSONNES')
plt.legend(title='COW (Occupation)')
plt.show()

print("")
print("DATABASE 2")
# print(labels.head())  # Show first lines from the dataset
print(labels.columns)   # Show all the columns on the dataset

In [3]:
print("\n-----------------------------")
print("    SHUFFLE AND DIVIDED DATA   ")
print("-----------------------------\n")
# Shuffle and Divide data
X_train, X_test, y_train, y_test = train_test_split(
    features,           # X_train - X_test Características (features)
    labels.values.ravel(),             # y_train - y_test  (labels)
    test_size=0.2,      # Size test set (20%)
    random_state=42,    # Para reproducibilidad
    shuffle=False       # Mix data TRUE
)

print("\nSize TRAINING set:", X_train.shape)
print("TRAINING set:")
print(X_train) # Print Results

print("\nSize TEST set:", X_test.shape, "\n")
print("TEST set:")
print(X_test) # Print Results

# To standaliser the datas
my_scaler = StandardScaler()
X_train_Standed = my_scaler.fit_transform(X_train.select_dtypes(include=['float64','int64']))
X_test_Standed = my_scaler.fit_transform(X_test.select_dtypes(include=['float64','int64']))
joblib.dump (my_scaler, 'my_scaler.joblib')

# from sklearn.utils import shuffle
datos_shuffled = shuffle(features)


-----------------------------
    SHUFFLE AND DIVIDED DATA   
-----------------------------


Size TRAINING set: (133052, 10)
TRAINING set:
        AGEP  COW  SCHL  MAR    OCCP   POBP  RELP  WKHP  SEX  RAC1P
0       41.0  4.0  24.0  1.0  2555.0    6.0   1.0  60.0  2.0    1.0
1       77.0  7.0  22.0  1.0  4920.0   39.0   0.0  35.0  1.0    1.0
2       38.0  1.0  18.0  1.0   440.0    6.0   1.0  50.0  1.0    1.0
3       30.0  1.0  22.0  5.0  1555.0    6.0   2.0  80.0  1.0    6.0
4       36.0  1.0  16.0  1.0  4030.0  314.0   1.0  70.0  2.0    1.0
...      ...  ...   ...  ...     ...    ...   ...   ...  ...    ...
133047  47.0  1.0  18.0  1.0  5240.0   47.0   0.0  40.0  2.0    2.0
133048  23.0  1.0  18.0  5.0  4720.0    6.0  13.0  30.0  2.0    1.0
133049  18.0  1.0  16.0  5.0  4220.0    6.0   2.0  10.0  1.0    8.0
133050  41.0  1.0  21.0  1.0  4850.0   29.0   1.0  30.0  1.0    1.0
133051  51.0  1.0  19.0  1.0  1010.0    6.0  15.0  40.0  1.0    1.0

[133052 rows x 10 columns]

Size TEST set:

In [ ]:
#SVM
# Validation Croisée
svm_model = SVC(kernel='linear')
scores = cross_val_score(svm_model,X_train,y_train,cv=5).mean()

# Accuracy & Classification report & Confusion matrix
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_svm)
classification_rep = classification_report(y_test, y_pred_svm)
conf_matrix = confusion_matrix(y_test, y_pred_svm)

# Grid Search for best parameters
param_grid = {'C': [0.1, 1, 10, 20], 'kernel': ['rbf', 'poly']}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

print("best_params = ", best_params)

In [5]:
# Models to be tested with params by default
models = {
    "RandomForest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
    "SVM": SVC()
}

# Cross-validation and initial metrics
for name, model in models.items():
    print(f"Training {name}...")
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    print(f"Accuracy average (Cross-Validation): {np.mean(scores):.4f}")
    
    # Train and evaluate on the test set
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"Accuracy on test set: {accuracy_score(y_test, y_pred):.4f}")
    print("Classification report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("-" * 40)

Training RandomForest...
Accuracy average (Cross Validation): 0.8126
Accuracy on test set: 0.8173
Classification report:
              precision    recall  f1-score   support

       False       0.84      0.86      0.85     19774
        True       0.78      0.76      0.77     13489

    accuracy                           0.82     33263
   macro avg       0.81      0.81      0.81     33263
weighted avg       0.82      0.82      0.82     33263

Confusion Matrix:
[[16933  2841]
 [ 3236 10253]]
----------------------------------------
Training AdaBoost...
Accuracy average (Cross Validation): 0.8042
Accuracy on test set: 0.8083
Classification report:
              precision    recall  f1-score   support

       False       0.83      0.85      0.84     19774
        True       0.77      0.75      0.76     13489

    accuracy                           0.81     33263
   macro avg       0.80      0.80      0.80     33263
weighted avg       0.81      0.81      0.81     33263

Confusion Matrix:


In [ ]:
# Hyperparameters for each model
param_grids = {
    "RandomForest": {"n_estimators": [50, 100, 200], "max_depth": [None, 10, 20]},
    "AdaBoost": {"n_estimators": [50, 100, 200], "learning_rate": [0.01, 0.1, 1]},
    "GradientBoosting": {"n_estimators": [50, 100, 200], "learning_rate": [0.01, 0.1, 1]},
    "SVM": {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]}
}

# Optimize each model
best_models = {}
for name, model in models.items():
    print(f"Optimización de hyperparameters for {name}...")
    grid = GridSearchCV(model, param_grids[name], cv=5, scoring='accuracy')
    grid.fit(X_train, y_train)
    best_models[name] = grid.best_estimator_
    
    print(f"Best Parameters for {name}: {grid.best_params_}")
    print(f"Best Accuracy (Cross-validation): {grid.best_score_:.4f}")

In [5]:
# Training SVM with the results of the base models.
# Training Models with the best hyperparameters
rf = RandomForestClassifier(n_estimators=200, max_depth=20)
gb = GradientBoostingClassifier(n_estimators=200, learning_rate=1)
ab = AdaBoostClassifier(n_estimators=200, learning_rate=1)

rf.fit(X_train, y_train)
gb.fit(X_train, y_train)
ab.fit(X_train, y_train)

# Generate predictions as features for the SVM - Train set
rf_preds = rf.predict_proba(X_train)[:, 1]
gb_preds = gb.predict_proba(X_train)[:, 1]
ab_preds = ab.predict_proba(X_train)[:, 1]

X_train_stack = np.column_stack((rf_preds, gb_preds, ab_preds))

# Generate predictions as features for the SVM - Test set
rf_preds_test = rf.predict_proba(X_test)[:, 1]
gb_preds_test = gb.predict_proba(X_test)[:, 1]
ab_preds_test = ab.predict_proba(X_test)[:, 1]

X_test_stack = np.column_stack((rf_preds_test, gb_preds_test, ab_preds_test))

# Training SVM lineal as meta-model
svm = LinearSVC(C=1, max_iter=10000, random_state=42)
svm.fit(X_train_stack, y_train)

# Evaluation
y_pred = svm.predict(X_test_stack)

print("Stacking model accuracy with SVM:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Stacking model accuracy with SVM: 0.792953131106635
              precision    recall  f1-score   support

       False       0.81      0.85      0.83     19774
        True       0.76      0.72      0.74     13489

    accuracy                           0.79     33263
   macro avg       0.79      0.78      0.78     33263
weighted avg       0.79      0.79      0.79     33263



In [ ]:
# Evaluate and save the best models
for name, model in best_models.items():
    print(f"Evaluation of the optimized model: {name}")
    y_pred = model.predict(X_test)
    
    print(f"Accuracy on test set: {accuracy_score(y_test, y_pred):.4f}")
    print("Classification report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    
    # Save Model
    filename = f"{name}_BestModel_{accuracy_score(y_test, y_pred):.4f}".replace(".", "") + ".joblib"
    joblib.dump(model, filename)
    print(f"Model saved as: {filename}")

In [ ]:


#Forest 

rf_model = RandomForestClassifier()
cv_scores_rf = cross_val_score(rf_model, X_train, y_train, cv=5).mean()
print("validation croisée average score = ", cv_scores_rf)


rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
classification_rep_rf = classification_report(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)


param_grid_rf = {'n_estimators': [50, 100, 120],'criterion':['gini','entropy','log_loss'],'max_depth': [None, 10, 20],'min_samples_split':[0.1, 1.0, 2]}
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5)
grid_search_rf.fit(X_train, y_train)
best_params_rf = grid_search_rf.best_params_




In [7]:
# Models to be tested with params by default
models2 = {
    "GradientBoosting": GradientBoostingClassifier(),
    "SVM": SVC()
}

In [ ]:
# Hiperparámetros para cada modelo
param_grids = {
    "GradientBoosting": {"n_estimators": [50, 100, 200], "learning_rate": [0.01, 0.1, 1]},
    "SVM": {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]}
}

# Optimizar cada modelo
best_models = {}
for name, model in models2.items():
    print(f"Optimización de hiperparámetros para {name}...")
    grid = GridSearchCV(model, param_grids[name], cv=5, scoring='accuracy')
    grid.fit(X_train, y_train)
    best_models[name] = grid.best_estimator_
    
    print(f"Mejores parámetros para {name}: {grid.best_params_}")
    print(f"Mejor precisión (validación cruzada): {grid.best_score_:.4f}")

Optimización de hiperparámetros para GradientBoosting...
